## Jakub Wiechnik

## HOMEWORK: $Z$ boson production in $p+\bar{p}$ collisions
### Generate 10000 $p+\bar{p}$ events at 630 GeV of $Z$ boson production ($Z$ discovery at CERN SPS). (in the center of mass)

Setup:
* in order to look at $Z$ boson production we limit physics list of Pythia8 to include only hard $f\bar{f}\to Z \to f\bar{f}$ processes
  * `WeakSingleBoson:ffbar2gmZ`
* moreover, we will disable all $Z$ decay modes other than $Z\to e^+e^-$, in which the particle was discovered
  * `id:onMode = off` disables all decay channels of particle with given id (`id` should be an integer equal to PDG code of the particle)
  * `id:onIfAny = prod` enables decay of particle `id` in channels containing `prod` - see pythia documentation if more info is necessary
* to speed up the simulation we disable hadronisation (we are interested only in leptonic final states)
* we also limit phase space to $\pm 20$ GeV around the $Z$ rest mass
  * use `PhaseSpace:mHatMin = XX` and `PhaseSpace:mHatMax = XX` options, values are in GeV

To do:
* select candidates for products of $Z$ decay in each event and plot their invariant mass distribution in range 70-110 GeV
* check influence of final state radiation on the spectrum - generate additional 10000 events with FSR turned off
* normalize histograms so that their maxima are at $1$
* draw both distributions (as a histograms, option `"hist"`) on the same plot with axis labels and a legend, how does FSR influence invariant mass spectrum?
* draw $\frac{1}{p_T}\frac{\mathrm{d}N}{\mathrm{d}p_T}$ and $\frac{\mathrm{d}N}{\mathrm{d}\eta}$ spectra of produced $Z$ bosons (only for one dataset, with FSR on), with axis labels and proper normalisation, pick the $x$ axis ranges so that the whole distribution fits inside the plot
  * **hint:** $Z$ boson will occur multiple times in the event listing (why?), it will never be a final particle, pick only $Z$'s that appear *last* in each event
* NOT COMPULSORY what is the *mean* number $\langle N_Z\rangle$ of $Z$ bosons produced in an event, with what error? is this result surprising? 
  * **hint:** $\langle N \rangle = \int\limits_0^\infty \frac{\mathrm{d}N}{\mathrm{d}p_T}\cdot \mathrm{d}p_T = \int\limits_{-\infty}^\infty \frac{\mathrm{d}N}{\mathrm{d}\eta}\cdot \mathrm{d}\eta$, see `TH1::Integral`, and `TH1::IntegralAndError` methods and make sure, that *"width"* option is used so that bin contents are multiplied by bin widths during computation of the integral

In [1]:
# Imported librareis
import ROOT
import os, sys, subprocess, time
import py8settings as py8s
from py8settings import PDF
import itertools as it
import ctypes
%jsroot on

Welcome to JupyROOT 6.24/04


Settings

In [2]:
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")
    
pythia = ROOT.TPythia8(False)
makePythiaSilent(pythia)
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 42")
pythia.ReadString("23:onMode = off")
pythia.ReadString("23:onIfAny = 11")
pythia.ReadString("HadronLevel:Hadronize = off")
pythia.ReadString("PhaseSpace:mHatMin = 70")
pythia.ReadString("PhaseSpace:mHatMax = 110")

# Warning about cross section violated is ok

Initialize our simulation

In [3]:
# FSR on
pythia.ReadString("PartonLevel:FSR = on")
particles = ROOT.TClonesArray("TParticle", 10000)
pythia.Initialize(2212 , -2212 , 630)
pythia.GenerateEvent()

Useful functions

In [4]:
def isFinal(p,name=None):
    if name==None:
        return p.GetStatusCode()>0
    else:
        return p.GetStatusCode()>0 and p.GetName()==name


def getMinv(pair):  # invariant mass of 2 particles
    p4_1=ROOT.TLorentzVector()  # ROOT class to represent 4vectors
    p4_2=ROOT.TLorentzVector()
    pair[0].Momentum(p4_1)
    pair[1].Momentum(p4_2)
    p4=p4_1+p4_2
    return p4.M()   # get mass of 4vector

## 1. FSR on 

In [5]:
#plotting invariant mass distribution of product of decay of Z boson (e+ and e-)
c1_kstar = ROOT.TCanvas("c1","",800,600)

nEvents=10000
h=ROOT.TH1D("h","",300,70,110)
parts=["e-", "e+"]
labels={"e-": "#e^{-}", "e+": "#e^{+}"}
colors={"e-": ROOT.kBlack, "e+": ROOT.kBlue}


def getZbosondecayCandidates(parts):
    electron = [p for p in parts if isFinal(p,"e-")]
    positron = [p for p in parts if isFinal(p,"e+")]
    return list(it.product(electron,positron))

for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    #for all possible e- e+ pairs we fill invariant mass histogram
    for pair in getZbosondecayCandidates(particles):
        h.Fill(getMinv(pair))
        
# Normalization
h.Scale(1/h.GetMaximum())

 PYTHIA Warning in JunctionSplitting::SplitJunPairs: parallel junction state not allowed.  
 PYTHIA Warning in JunctionSplitting::CheckColours: Not possible to split junctions; making new colours  
 PYTHIA Error in BeamRemnants::add: failed to find physical colour state after colour reconnection  
 PYTHIA Warning in MultipartonInteractions::pTnext: weight above unity  


Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG


Make plot with fsr on

In [6]:
h.GetXaxis().SetTitle("M_{#e^{-}e^{+}}[GeV/#it{c}^{2}]")
h.GetXaxis().SetTitleOffset(1.1)
h.GetYaxis().SetTitle("Entries")
h.SetLineColor(ROOT.kBlue)
h.SetLineWidth(2)
h.DrawCopy()


## 2. FSR off

Settings

In [7]:
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")
    
pythia = ROOT.TPythia8(False)
makePythiaSilent(pythia)
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 42")
pythia.ReadString("23:onMode = off")
pythia.ReadString("23:onIfAny = 11")
pythia.ReadString("HadronLevel:Hadronize = off")
pythia.ReadString("PhaseSpace:mHatMin = 70")
pythia.ReadString("PhaseSpace:mHatMax = 110")

# Warning about cross section violated is ok

Initialization

In [8]:
# FSR off
pythia.ReadString("PartonLevel:FSR = off")
particles = ROOT.TClonesArray("TParticle", 10000)
pythia.Initialize(2212 , -2212 , 630)
pythia.GenerateEvent()

 *** TDatabasePDG::AddParticle: particle with PDGcode=90 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900210 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900220 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900330 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900440 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902210 already defined


In [9]:
#plotting invariant mass distribution of product of decay of Z boson (e+ and e-)
nEvents=10000
hfsr=ROOT.TH1D("hfsr","",300,70,110)
parts=["e-", "e+"]
labels={"e-": "#e^{-}", "e+": "#e^{+}"}

def getZbosondecayCandidates(parts):
    electron = [p for p in parts if isFinal(p,"e-")]
    positron = [p for p in parts if isFinal(p,"e+")]
    return list(it.product(electron,positron))

for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    #for all possible e- e+ pairs we fill invariant mass histogram
    for pair in getZbosondecayCandidates(particles):
        hfsr.Fill(getMinv(pair))
        
# Normalization
hfsr.Scale(1/hfsr.GetMaximum())

 PYTHIA Warning in JunctionSplitting::SplitJunPairs: parallel junction state not allowed.  
 PYTHIA Warning in JunctionSplitting::CheckColours: Not possible to split junctions; making new colours  
 PYTHIA Error in BeamRemnants::add: failed to find physical colour state after colour reconnection  
 PYTHIA Warning in SimpleSpaceShower::pT2nextQCD: weight above unity  
 PYTHIA Warning in MultipartonInteractions::pTnext: weight above unity  
 PYTHIA Error in SimpleSpaceShower::pT2nearThreshold: stuck in loop  


Plotting final

In [10]:
hfsr.GetXaxis().SetTitle("M_{#e^{-}e^{+}}[GeV/#it{c}^{2}]")
hfsr.GetXaxis().SetTitleOffset(1.1)
hfsr.GetYaxis().SetTitle("Entries")
hfsr.SetLineColor(ROOT.kGreen)
hfsr.SetLineWidth(2)
hfsr.DrawCopy("same")

# Draw final plot and

leg=ROOT.TLegend(0.6,0.8,0.9,0.9)
leg.AddEntry(h,"FSR on e^{-}e^{+} pairs")
leg.AddEntry(hfsr,"FSR off e^{-}e^{+} pairs")
leg.Draw()

c1_kstar.Update()
c1_kstar.Draw()

## Spectra for fsr on

In [11]:
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")
    
pythia = ROOT.TPythia8(False)
makePythiaSilent(pythia)
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 42")
pythia.ReadString("23:onMode = off")
pythia.ReadString("23:onIfAny = 11")
pythia.ReadString("HadronLevel:Hadronize = off")
pythia.ReadString("PhaseSpace:mHatMin = 70")
pythia.ReadString("PhaseSpace:mHatMax = 110")

# Warning about cross section violated is ok

In [12]:
# FSR on
pythia.ReadString("PartonLevel:FSR = on")
particles = ROOT.TClonesArray("TParticle", 10000)
pythia.Initialize(2212 , -2212 , 630)
pythia.GenerateEvent()

 *** TDatabasePDG::AddParticle: particle with PDGcode=90 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900210 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900220 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900330 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900440 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902210 already defined


In [13]:
nEvents=10000
parts=["e-", "e+"]
labels={"e-": "#e^{-}", "e+": "#e^{+}"}
colors={"e-": ROOT.kBlack, "e+": ROOT.kBlue}

heta={}
hpt={}

for p in parts:
    heta[p]=ROOT.TH1D("eta"+p,"",120,-12,12)
    heta[p].Sumw2()
    hpt[p]=ROOT.TH1D("pt"+p,"",50,0,10)
    hpt[p].Sumw2()

def fillHistos(p):
    name=p.GetName()
    if name in parts:
        heta[name].Fill(p.Eta())
        hpt[name].Fill(p.Pt(),1/(p.Pt()))

#generate events
for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    for p in particles:
        if isFinal(p):
            fillHistos(p)
#format and normalize histograms, find ranges on y axes
maxeta=0
maxpt=0
for p in parts:
    heta[p].SetMarkerStyle(ROOT.kFullCircle)
    heta[p].SetMarkerColor(colors[p])
    bw=heta[p].GetBinWidth(1)
    heta[p].Scale(1./(bw*nEvents))
    maxeta=max(maxeta,heta[p].GetMaximum())
    
    hpt[p].SetMarkerStyle(ROOT.kFullCircle)
    hpt[p].SetMarkerColor(colors[p])
    bw=hpt[p].GetBinWidth(1)
    hpt[p].Scale(1./(bw*nEvents))
    maxpt=max(maxpt,hpt[p].GetMaximum())

 PYTHIA Warning in JunctionSplitting::SplitJunPairs: parallel junction state not allowed.  
 PYTHIA Warning in JunctionSplitting::CheckColours: Not possible to split junctions; making new colours  
 PYTHIA Error in BeamRemnants::add: failed to find physical colour state after colour reconnection  
 PYTHIA Warning in MultipartonInteractions::pTnext: weight above unity  


In [14]:
c_etapt = ROOT.TCanvas("c_etapt","",1600,600)
c_etapt.Divide(2,1)

legend=ROOT.TLegend(0.8,0.75,0.9,0.9)
for p in parts:
    legend.AddEntry(hpt[p], labels[p])

c_etapt.cd(1).SetLogy()
frame_pt=c_etapt.cd(1).DrawFrame(0,maxpt*1e-8,10,10*maxpt)
frame_pt.GetXaxis().SetTitle("p_{T} [GeV/c]")
frame_pt.GetXaxis().SetTitleOffset(1.5)
frame_pt.GetYaxis().SetTitleSize(15)
frame_pt.GetYaxis().SetTitle("#frac{1}{p_{T}}#frac{dN}{dp_{T}} [GeV/c]^{-2}")
frame_pt.Draw()
for p in parts:
    hpt[p].Draw("p same")
legend.Draw()

c_etapt.cd(2)
frame_eta=c_etapt.cd(2).DrawFrame(-12,0,12,1.1*maxeta)
frame_eta.GetXaxis().SetTitle("#eta")
frame_eta.GetYaxis().SetTitleSize(15)
frame_eta.GetYaxis().SetTitle("#frac{dN}{d#eta} ")
frame_eta.Draw()
for p in parts:
    heta[p].Draw("p same")
    
legend.Draw()

c_etapt.Draw()